In [4]:
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    "--repositories http://repo.hortonworks.com/content/groups/public/ "
    "--packages com.hortonworks:shc-core:1.1.1-2.1-s_2.11 "
    " pyspark-shell")
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [30]:
sc = SparkContext()
sqlc = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-1a6cca07cc47>:1 

## Data Analysis using pyspark.sql

In [62]:
data_source_format = 'org.apache.spark.sql.execution.datasources.hbase'

df = sc.parallelize([('a', '1.0'), ('b', '2.0')]).toDF(schema=['col0', 'col1'])

# ''.join(string.split()) in order to write a multi-line JSON string here.
import json
catalog = json.dumps({
    "table":{"namespace":"default", "name":"20181205_213511_alexdziena_tweets"},
    "rowkey":"row_id",
    "columns":{
        "row_id":{"cf":"rowkey", "col":"row_id", "type":"string"},
        "category":{"cf":"tweet", "col":"category", "type":"string"},
        "contributors":{"cf":"tweet", "col":"contributors", "type":"string"},
        "coordinates":{"cf":"tweet", "col":"coordinates", "type":"string"},
        "created_at":{"cf":"tweet", "col":"created_at", "type":"string"},
        "display_text_range":{"cf":"tweet", "col":"display_text_range", "type":"string"},
        "entities":{"cf":"tweet", "col":"entities", "type":"string"},
        "extended_tweet":{"cf":"tweet", "col":"extended_tweet", "type":"string"},
        "favorite_count":{"cf":"tweet", "col":"favorite_count", "type":"int"},
        "favorited":{"cf":"tweet", "col":"favorited", "type":"boolean"},
        "filter_level":{"cf":"tweet", "col":"filter_level", "type":"string"},
        "geo":{"cf":"tweet", "col":"geo", "type":"string"},
        "id":{"cf":"tweet", "col":"id", "type":"bigint"},
        "in_reply_to_screen_name":{"cf":"tweet", "col":"in_reply_to_screen_name", "type":"string"},
        "in_reply_to_status_id":{"cf":"tweet", "col":"in_reply_to_status_id", "type":"string"},
        "in_reply_to_status_id_str":{"cf":"tweet", "col":"in_reply_to_status_id_str", "type":"string"},
        "in_reply_to_user_id":{"cf":"tweet", "col":"in_reply_to_user_id", "type":"string"},
        "in_reply_to_user_id_str":{"cf":"tweet", "col":"in_reply_to_user_id_str", "type":"string"},
        "is_quote_status":{"cf":"tweet", "col":"is_quote_status", "type":"boolean"},
        "lang":{"cf":"tweet", "col":"lang", "type":"string"},
        "place":{"cf":"tweet", "col":"place", "type":"string"},
        "quote_count":{"cf":"tweet", "col":"quote_count", "type":"int"},
        "reply_count":{"cf":"tweet", "col":"reply_count", "type":"int"},
        "retweet_count":{"cf":"tweet", "col":"retweet_count", "type":"int"},
        "retweeted":{"cf":"tweet", "col":"retweeted", "type":"boolean"},
        "source":{"cf":"tweet", "col":"source", "type":"string"},
        "text":{"cf":"tweet", "col":"text", "type":"string"},
        "timestamp_ms":{"cf":"tweet", "col":"timestamp_ms", "type":"bigint"},
        "truncated":{"cf":"tweet", "col":"truncated", "type":"boolean"}, 
    }
})


# Writing
#df.write\
#.options(catalog=catalog)\  # alternatively: .option('catalog', catalog)
#.format(data_source_format)\
#.save()

# Reading
df = sqlc.read\
.options(catalog=catalog)\
.format(data_source_format)\
.load()

df.selectExpr(
    "row_id",
    "contributors",
    "geo",
    "created_at",
    "category",
    "text",
    "favorite_count",
    "reply_count",
    "retweet_count",
    "quote_count",
    "retweeted"
).limit(2).show()

+-------------------+------------+----+--------------------+--------+--------------------+--------------+-----------+-------------+-----------+---------+
|             row_id|contributors| geo|          created_at|category|                text|favorite_count|reply_count|retweet_count|quote_count|retweeted|
+-------------------+------------+----+--------------------+--------+--------------------+--------------+-----------+-------------+-----------+---------+
|1070506938381860864|        None|None|Thu Dec 06 02:35:...|     mvg|I can’t imagine a...|             0|          0|            0|          0|    false|
|1070506947407794177|        None|None|Thu Dec 06 02:35:...|     mvg|RT @NikileshSurya...|             0|          0|            0|          0|    false|
+-------------------+------------+----+--------------------+--------+--------------------+--------------+-----------+-------------+-----------+---------+



In [70]:
sqlc.registerDataFrameAsTable(df, "table1")
df2 = sqlc.sql("SELECT  text, reply_count, retweet_count from table1")
df2.collect()

[Row(text='I can’t imagine a game launch worse than Fallout 76, holy shit it just keeps getting worse', reply_count=0, retweet_count=0),
 Row(text='RT @NikileshSurya: Game on as #Viswasam takes centre stage @RohiniSilverScr  and confirms release !! #viswasamதிருவிழாatrohini #VISWASAM ht…', reply_count=0, retweet_count=0),
 Row(text='RT @NikileshSurya: Game on as #Viswasam takes centre stage @RohiniSilverScr  and confirms release !! #viswasamதிருவிழாatrohini #VISWASAM ht…', reply_count=0, retweet_count=0),
 Row(text="RT @OMGitsAliA: In (just over) 1 year of Fortnite: BR's release\n\n- 50+ updates \n- 7 Seasons\n- Evolved map\n- $10,000,000+ into eSports prize…", reply_count=0, retweet_count=0),
 Row(text='RT @NikileshSurya: Game on as #Viswasam takes centre stage @RohiniSilverScr  and confirms release !! #viswasamதிருவிழாatrohini #VISWASAM ht…', reply_count=0, retweet_count=0),
 Row(text='RT @NikileshSurya: Game on as #Viswasam takes centre stage @RohiniSilverScr  and confirms release !

In [53]:
df.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- contributors: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: string (nullable = true)
 |-- entities: string (nullable = true)
 |-- extended_tweet: string (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- favorited: boolean (nullable = true)
 |-- filter_level: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- id: long (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |-- in_reply_to_user_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- is_quote_status: boolean (nullable = true)
 |-- lang: string (nullable = true)
 |-- place: string (nullable = true)
 |-- quote_count: integer (nullable = true)
 |-- reply_count

In [44]:
import os
import findspark
findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, rand
from enum import Enum

conf = SparkConf()
conf.setAppName("generate_graph")
#sc = SparkContext(conf=conf)


In [56]:
df.sql("SELECT * FROM table")

AttributeError: 'DataFrame' object has no attribute 'sql'